<a href="https://colab.research.google.com/github/hschilling/google_colab/blob/master/openmdao_pandas_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openmdao

In [0]:
import pandas as pd

In [0]:
import cufflinks as cf

In [4]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

driver.add_recorder(om.SqliteRecorder("cases.sql"))

driver.recording_options['includes'] = []
driver.recording_options['record_objectives'] = True
driver.recording_options['record_constraints'] = True
driver.recording_options['record_desvars'] = True

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")


Unable to import mpi4py. Parallel processing unavailable.


In [0]:
def case_recorder_outputs_to_data_frame(cr):
  case = cr.get_case(0)
  outputs = case.list_outputs(out_stream=None)
  varnames = [out[0] for out in outputs]
  data = {}
  for i in range(len(cr.list_cases())):
    case = cr.get_case(i)
    for varname in varnames:
      if case[varname].shape == (1,):
        val = case[varname][0]
      else:
        val = np.linalg.norm(case[varname][0])
      if varname not in data:
        data[varname] = []
      data[varname].append(val)
  df = pd.DataFrame(data)
  return df

In [0]:
df_from_cr = case_recorder_outputs_to_data_frame(cr)

In [0]:
fig = df_from_cr[['con_cmp1.con1','px.x']].iplot(asFigure=True)


In [8]:
fig.show()